# In this article, we will look at some common techniques to structure your CNN model

In [ ]:
import tensorflow as tf
import keras
from keras.callbacks import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import *

In [ ]:
traindata = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
trainlabel=traindata['label'].values
traindata.drop('label',inplace=True,axis=1)
trainimages = traindata.values
trainimages=trainimages.reshape(-1,28,28,1)

In [ ]:
testdata = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')
testlabel=testdata['label'].values
testdata.drop('label',inplace=True,axis=1)
testimages = testdata.values
testimages=testimages.reshape(-1,28,28,1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
traingen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2)
traindata_generator = traingen.flow(trainimages,trainlabel,batch_size=64, subset='training')
validationdata_generator = traingen.flow(trainimages,trainlabel,batch_size=64,subset='validation')

In [ ]:
testgen=ImageDataGenerator(rescale=1/255.0)
testdata_generator = testgen.flow(testimages,testlabel)

In [ ]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(28,28,1)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(25,activation="softmax"))


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
history0=model.fit(traindata_generator,epochs=15,validation_data=validationdata_generator)

In [ ]:
acc = history0.history['accuracy']
val_acc = history0.history['val_accuracy']
loss = history0.history['loss']
val_loss = history0.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
print("test accuracy: "+ str(model.evaluate_generator(testdata_generator)[1]*100))

#### Training and Validation Accuracy: 100
#### Test Accuracy: 93.2
#### Problem: Overfitting Solution: Data Augmentation

# Model Using Data Augmentation

In [ ]:
traingen=ImageDataGenerator(rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1,
                  shear_range=0.1,horizontal_flip=True,rescale=1/255.0,validation_split=0.2)
traindata_generator = traingen.flow(trainimages,trainlabel,batch_size=128,subset='training')
validationdata_generator = traingen.flow(trainimages,trainlabel,batch_size=128,subset='validation')

In [ ]:
testgen=ImageDataGenerator(rescale=1/255.0)
testdata_generator = testgen.flow(testimages,testlabel)

In [ ]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(28,28,1)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(25,activation="softmax"))


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
history1=model.fit(traindata_generator,epochs=40,validation_data=validationdata_generator)

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
print("test accuracy: "+ str(model.evaluate_generator(testdata_generator)[1]*100))

### Train Accuracy: 98.2 Test Accuracy: 98.5
### If we train for more epochs, the accuracy will increase again and loss will converge. But is there any Solution, to achieve same accuracy in less epoch.
### Yes, Batch Normalisation allows to train the same model in less epoch


# Batch Normalisation

In [ ]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))


model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(25,activation="softmax"))


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
history2=model.fit(traindata_generator,epochs=40,validation_data=validationdata_generator)

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
print("test accuracy: "+ str(model.evaluate_generator(testdata_generator)[1]*100))

### Training Accuracy:99.5 Test Accuracy: 98.3
### But the validation loss graph has not converged as compared Training loss, this means there is uncertainity in the model.
### Lets run it again to understand what exactly it is

In [ ]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))


model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(25,activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
history3=model.fit(traindata_generator,epochs=40,validation_data=validationdata_generator)

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
print("test accuracy: "+ str(model.evaluate_generator(testdata_generator)[1]*100))

### We can see there is high variance in validation and test accuracy, based on where we end up in last epoch, the accuracy for the model may be great or worse. 

### This is happening because of large learning rate which is causing to overshoot the optima during last phases of training. We can use Decaying Learning Rate, which reduces learning rate after each epoch, hence allowing the graph to converge more smoothly.

# Decaying Learning Rate

In [ ]:
model=Sequential([])

model.add(Conv2D(64,(3,3),activation="relu",input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))


model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(25,activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
decaylr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [ ]:
history4=model.fit(traindata_generator,epochs=40,validation_data=validationdata_generator,callbacks=[decaylr])

In [ ]:
acc = history4.history['accuracy']
val_acc = history4.history['val_accuracy']
loss = history4.history['loss']
val_loss = history4.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### All the curves have converged smoothly by end of 15 epochs and hence there is high confidence in the model.

In [ ]:
print("test accuracy: "+ str(model.evaluate_generator(testdata_generator)[1]*100))

### Test Accuracy is 100%,(though it  might not be the case in every run but it will be almost close to 100% in every run.

### P.S: There's no need to train the model for 40 epochs, we can use callback to stop training once we reached desired accuracy